In [1]:
from neat2048.refactor.game import Game2048, UP, DOWN, LEFT, RIGHT
import pickle
from neat2048.refactor.hyper import HyperNetwork
from neat2048.refactor.nets import get_layers_descriptors
from neat2048.refactor.game_fitnesses import GameScoreFitness, GameStats
from neat2048.refactor.predictions import MinMaxNetPredictor, Log2ScaledConverter
from neat2048.refactor.nets import HyperNetCreator

cppn = pickle.load(open("output_network.pkl", "rb"))

board_size_x, board_size_y = 4, 4


min_max_predictor = MinMaxNetPredictor(
    converter=Log2ScaledConverter(),
    depth=2,
)

net_creator = HyperNetCreator()

descriptors = get_layers_descriptors(board_size_x, board_size_y)


def get_weights(inputs: list[float]) -> list[float]:
    return cppn.activate(inputs)


net = HyperNetwork(get_layers_descriptors(board_size_x, board_size_y), get_weights)

In [9]:
from IPython.display import clear_output

game = Game2048(size_x=board_size_x, size_y=board_size_y)
game.add_random_tile()

print(repr(game))


def get_pretty_move(move: int) -> str:
    if move == UP:
        return "UP"
    elif move == DOWN:
        return "DOWN"
    elif move == LEFT:
        return "LEFT"
    elif move == RIGHT:
        return "RIGHT"
    else:
        return "UNKNOWN"


while not game.game_end:
    # _press = input("Press enter to continue")
    clear_output()

    moves = min_max_predictor.predict_batch(net, [game])[0]

    for move, _ in sorted(moves, key=lambda x: x[1], reverse=True):
        changed = game.move(move, add_random_tile=True)
        print("Move: ", get_pretty_move(move))
        print("Score: ", game.score)
        if changed:
            break

    print(repr(game))

print(game.score)

Move:  UP
Score:  1868
             
 8  2   4 32 
             
 4 32  64  2 
             
 2 64 128 16 
             
 4  8  16 32 
             

1868


In [3]:
print(GameScoreFitness(scale=False).fitness(GameStats(game, 0, 0, 0)))

11.165535141377925
